In [1]:
from data_loader import get_custom_data
import pandas as pd
import json
import os

In [4]:
%%time
get_custom_data(None, data = 'FashionGen', preprocess_fn = None, is_train = True, cls = False, subclass = False)

CPU times: user 1min 50s, sys: 1.15 s, total: 1min 51s
Wall time: 1min 51s


In [5]:
%%time
get_custom_data(None, data = 'FashionGen', preprocess_fn = None, is_train = True, cls = True, subclass = False)

AttributeError: 'FashionGen' object has no attribute 'classes'

In [2]:
os.listdir('./data')

['sydney_captions',
 'scicap_data.zip',
 'polyvore',
 'scicap',
 'fashion200k',
 'UCM_captions.zip',
 'RSICD.zip',
 'polyvore-images.tar.gz',
 'RSICD',
 'UCM',
 'UCMerced_LandUse',
 'WHU-RS19',
 'RSSCN7',
 'AID',
 'NWPU-RESISC45',
 'kaggle_simpsons_characters.zip',
 'kaggle_simpsons_characters',
 'polyvore_outfits.zip',
 'polyvore_outfits']

In [5]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = False, subclass = False)

/vol/tensusers5/nhollain/data_loader.py:226: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  category_dict = category_df[['ID', 'subcategory']].set_index('ID').T.to_dict('records')


CPU times: user 7min 11s, sys: 151 ms, total: 7min 11s
Wall time: 7min 11s


In [6]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = True, subclass = False)

/vol/tensusers5/nhollain/data_loader.py:226: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  category_dict = category_df[['ID', 'subcategory']].set_index('ID').T.to_dict('records')


CPU times: user 5min 50s, sys: 172 ms, total: 5min 50s
Wall time: 5min 51s


(<data_loader.Polyvore at 0x14f0ff8f4ca0>,
 ['accessories',
  'all-body',
  'bags',
  'bottoms',
  'hats',
  'jewellery',
  'outerwear',
  'scarves',
  'shoes',
  'sunglasses',
  'tops'],
 [<function data_loader.get_custom_data.<locals>.<lambda>(c)>])

In [3]:
# PROBLEMS WITH POLYVORE DATA: the data structure seems to be different from what the S-CLIP paper uses.
# I want to probably get the dict['items'] values for each folder and then take the images from there.
working_datasets = ['RSICD', 'UCM', 'Sydney', 'Fashion200k', 'SciCap', 'RSICD-CLS', 'UCM-CLS', 'Fashion200k-CLS', 'WHU-RS19', 'RSSCN7', 
                    'AID', 'RESISC45', 'Simpsons-Images', 'Polyvore', 'Polyvore-CLS']
# get_custom_data(args, data, preprocess_fn, is_train, cls, subclass, **data_kwargs):
for dataset in working_datasets:
    print(dataset)
    cls = 'CLS' in dataset
    # CLS version of most datasets just uses the non-CLS dataloader, but adds cls = True as argument
    if dataset != 'UCM-CLS': # Exception: UCM-CLS is a different dataloader altogether than UCM!
        dataset = dataset.replace('-CLS', '')
    get_custom_data(None, data = dataset, preprocess_fn = None, is_train = True, cls = cls, subclass = False)

RSICD
UCM
Sydney
Fashion200k
SciCap
RSICD-CLS
UCM-CLS
Fashion200k-CLS
WHU-RS19
RSSCN7
AID
RESISC45


In [1]:
get_custom_data(None, data = 'Simpsons-Images', preprocess_fn = None, is_train = True, cls = True, subclass = False)

Dataset ImageFolder
    Number of datapoints: 20933
    Root location: ./data/kaggle_simpsons_characters/simpsons_dataset